In [0]:
# Importando bibliotecas necessárias
from pyspark.sql.functions import col, to_date, trim

# --- Configuração dos Caminhos ---

storage_account_name = "stacdatatrabalhoed01"

bronze_path = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net"
silver_path = f"abfss://silver@{storage_account_name}.dfs.core.windows.net"

Processando Pacientes...
Processando Consultas...
Processando tabelas simples...
Processo de transformação para a camada Silver concluído.


In [0]:
# --- Lendo as tabelas da camada Bronze ---
df_paciente_bronze = spark.read.format("delta").load(f"{bronze_path}/paciente")
df_endereco_bronze = spark.read.format("delta").load(f"{bronze_path}/endereco")
df_agendamento_bronze = spark.read.format("delta").load(f"{bronze_path}/agendamento")
df_consulta_bronze = spark.read.format("delta").load(f"{bronze_path}/consulta")
df_pagamento_bronze = spark.read.format("delta").load(f"{bronze_path}/pagamento")
df_tipo_pagamento_bronze = spark.read.format("delta").load(f"{bronze_path}/tipo_pagamento")
df_log_pagamento_bronze = spark.read.format("delta").load(f"{bronze_path}/log_pagamento")
df_procedimento_bronze = spark.read.format("delta").load(f"{bronze_path}/procedimento")
df_odontologista_bronze = spark.read.format("delta").load(f"{bronze_path}/odontologista")
df_consulta_procedimento_bronze = spark.read.format("delta").load(f"{bronze_path}/consulta_procedimento")

In [0]:
# --- Transformação 1: Paciente + Endereço ---
print("Processando Pacientes...")
df_paciente_silver = df_paciente_bronze.join(
    df_endereco_bronze,
    df_paciente_bronze["endereco_id_endereco"] == df_endereco_bronze["id_endereco"],
    "left"  # Usamos left join para não perder pacientes sem endereço cadastrado
).select(
    df_paciente_bronze["id_paciente"],
    df_paciente_bronze["nome_paciente"],
    df_paciente_bronze["cpf_paciente"],
    df_paciente_bronze["telefone"],
    df_paciente_bronze["genero"],
    to_date(col("data_nasc"), "yyyy-MM-dd").alias("data_nasc"),
    df_paciente_bronze["email"],
    df_endereco_bronze["logradouro"],
    df_endereco_bronze["numero"],
    df_endereco_bronze["complemento"],
    df_endereco_bronze["bairro"],
    df_endereco_bronze["cidade"],
    df_endereco_bronze["estado"],
    df_endereco_bronze["cep"],
    df_endereco_bronze["pais"]
)

# Salvar a tabela de paciente enriquecida na Silver
df_paciente_silver.write.format("delta").mode("overwrite").save(f"{silver_path}/paciente_enriquecido")
spark.sql(f"CREATE TABLE IF NOT EXISTS silver_paciente USING DELTA LOCATION '{silver_path}/paciente_enriquecido'")

In [0]:
# --- Transformação 2: Visão Consolidada da Consulta ---
# Unir consulta, agendamento e pagamento para ter uma visão mais completa
print("Processando Consultas...")
df_consulta_silver = df_consulta_bronze.join(
    df_agendamento_bronze,
    df_consulta_bronze["agendamento_id_agendamento"] == df_agendamento_bronze["id_agendamento"],
    "inner" # Inner join pois uma consulta deve ter um agendamento
).join(
    df_pagamento_bronze,
    df_consulta_bronze["id_consulta"] == df_pagamento_bronze["consulta_id_consulta"],
    "left" # Left join pois uma consulta pode não ter um pagamento ainda
).select(
    df_consulta_bronze["id_consulta"],
    df_consulta_bronze["data_hora"],
    df_consulta_bronze["diagnostico"],
    df_consulta_bronze["tratamento"],
    df_agendamento_bronze["id_agendamento"],
    df_agendamento_bronze["paciente_id_paciente"],
    df_agendamento_bronze["odontologista_id_odontologista"],
    df_agendamento_bronze["status_agendamento"],
    df_pagamento_bronze["id_pagamento"],
    df_pagamento_bronze["valor_pago"],
    df_pagamento_bronze["data_pagamento"],
    df_pagamento_bronze["tipo_pagamento_id_tipo_pagamento"]
)

df_consulta_silver.write.format("delta").mode("overwrite").save(f"{silver_path}/consulta_consolidada")
spark.sql(f"CREATE TABLE IF NOT EXISTS silver_consulta USING DELTA LOCATION '{silver_path}/consulta_consolidada'")


# As outras tabelas que não precisam de joins complexos (odontologista, procedimento, etc.)
# podem ser simplesmente limpas (ex: trim, cast) e salvas na Silver.
print("Processando tabelas simples...")
tabelas_simples = ["odontologista", "procedimento", "tipo_pagamento", "consulta_procedimento",]
for table_name in tabelas_simples:
    df_bronze = spark.read.format("delta").load(f"{bronze_path}/{table_name}")
    df_bronze.write.format("delta").mode("overwrite").save(f"{silver_path}/{table_name}")
    spark.sql(f"CREATE TABLE IF NOT EXISTS silver_{table_name} USING DELTA LOCATION '{silver_path}/{table_name}'")

print("Processo de transformação para a camada Silver concluído.")